<a href="https://colab.research.google.com/github/tip0x/MachineLearning/blob/main/Brain_Tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BRAIN TUMOR IMAGE CLASSIFICATION USING KERAS MODEL


This code below mounts google drive to enable cloud communication with the framework.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

This code below uses the tf.config.experimental module to configure GPU memory growth in TensorFlow 2.
By enabling GPU memory growth, TensorFlow will initially allocate a small amount of GPU memory and will dynamically increase memory allocation when necessary to avoid situation where the entire GPU memory is locked up by one process, allowing other processes or tasks to use the remaining memory.

In [ ]:
import tensorflow as tf

# Set GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)



 This code below imports necessary libraries and modules for working with image data. It's a typical setup for building and training image classification models using TensorFlow and Keras.

In [ ]:
!pip install split-folders
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import InceptionV3, VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [ ]:
import random

seed_value = 42
random.seed(seed_value)

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = "/content/drive/MyDrive/ML/Brain_Tumor_MRI_Classification/Training"
valid_path = "/content/drive/MyDrive/ML/Brain_Tumor_MRI_Classification/Testing"

In [ ]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
for layer in inception.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/ML/Brain_Tumor_MRI_Classification/Training/*')

In [ ]:
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale= 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/ML/Brain_Tumor_MRI_Classification/Training',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2870 images belonging to 4 classes.


In [ ]:
testing_set = test_datagen.flow_from_directory('/content/drive/MyDrive/ML/Brain_Tumor_MRI_Classification/Testing',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 394 images belonging to 4 classes.


In [ ]:
no_classes=len(training_set)
print(no_classes)

90


In [ ]:
r = model.fit(
    training_set,
    validation_data=testing_set,
    epochs=35,
    steps_per_epoch=len(training_set),
    validation_steps=len(testing_set)
)

Epoch 1/35
90/90 [==============================] - 1083s 12s/step - loss: 2.8028 - accuracy: 0.6990 - val_loss: 3.3651 - val_accuracy: 0.6218
Epoch 2/35
90/90 [==============================] - 436s 5s/step - loss: 0.9191 - accuracy: 0.8369 - val_loss: 2.3008 - val_accuracy: 0.6802
Epoch 3/35
90/90 [==============================] - 395s 4s/step - loss: 1.0806 - accuracy: 0.8387 - val_loss: 3.7305 - val_accuracy: 0.7056
Epoch 4/35
90/90 [==============================] - 431s 5s/step - loss: 1.5133 - accuracy: 0.8240 - val_loss: 3.9106 - val_accuracy: 0.7310
Epoch 5/35
90/90 [==============================] - 433s 5s/step - loss: 0.7637 - accuracy: 0.8791 - val_loss: 2.4949 - val_accuracy: 0.7665
Epoch 6/35
90/90 [==============================] - 429s 5s/step - loss: 0.6474 - accuracy: 0.9038 - val_loss: 3.9360 - val_accuracy: 0.7284
Epoch 7/35
88/90 [============================>.] - ETA: 7s - loss: 0.5918 - accuracy: 0.9145 

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_loss')

In [ ]:
from tensorflow.keras.models import load_model

model.save('model_brain_tumor.h5')

In [ ]:
!pip install -q gradio
!pip install -q tensorflow


In [ ]:
import gradio as gr
import requests
import urllib
import tensorflow as tf

model = tf.keras.models.load_model('/content/model_brain_tumor.h5')

labels = ['pituitary_tumor', 'no_tumor', 'meningioma_tumor', 'glioma_tumor']

#Add Batch Dimension

def classify_image(inp):
  inp = inp[None, ...]
  inp = tf.keras.applications.inception_v3.preprocess_input(inp)
  prediction = model.predict(inp).flatten()
  return{labels[i]: float(prediction[i])for i in range(no_classes)}

  #Create the gradio interface

image = gr.inputs.Image(shape = (224, 224))
label = gr.outputs.Label(num_top_classes=4)

gr.Interface(fn = classify_image, inputs=image, outputs=label, interpretation = "default").launch(share=True)
